<a href="https://colab.research.google.com/github/LarsAhrenberg/DigitalOcean/blob/master/transformerintro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 34.9 MB/s 
     |████████████████████████████████| 596 kB 14.7 MB/s 
     |████████████████████████████████| 77 kB 3.5 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import transformers

In [3]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.2 MB 8.1 MB/s 


In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [5]:
classifier("Most people could complete the course.")

[{'label': 'POSITIVE', 'score': 0.992240309715271}]

In [6]:
generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you",
    max_length=30,
    num_return_sequences=2,
)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you the full fundamentals of human behavior. Then we will take those lessons and do them with you.\n\nThis'},
 {'generated_text': 'In this course, we will teach you how to develop a simple and accurate strategy for running, and more specifically – learning about managing and using your own'}]

In [7]:
generator(
    "While she had had every opportunity to change",
    max_length=30,
    num_return_sequences=3,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'While she had had every opportunity to change her mind, the old man made her feel like something else she wanted to change. As it was, the'},
 {'generated_text': 'While she had had every opportunity to change the subject and talk privately about it, many said she had always had no desire to remain silent.\n\n'},
 {'generated_text': 'While she had had every opportunity to change her name and be one of her own. “”\n\nOne day she met another man.'}]

In [9]:
import pandas as pd
ner_tagger = pipeline("ner", aggregation_strategy="simple")
text = """ My sister lives in London. Her name is Abigail. She bought a Jaguar two months ago. I love her so."""
output = ner_tagger(text)
pd.DataFrame(output)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


,entity_group,score,word,start,end
0,LOC,0.999279,London,20,26
1,PER,0.996502,Abigail,40,47
2,ORG,0.859182,Jaguar,62,68


In [15]:
from transformers import AutoModel,AutoTokenizer

In [16]:
expert = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
question = "Where does my sister live?"
output = expert(question=question, context=text)
print(output)

{'score': 0.9876561760902405, 'start': 20, 'end': 26, 'answer': 'London'}


In [18]:
def retokenize (txt):
  l = []
  for token in txt:
    if token['word'].startswith('##'):
        l[-1]['word'] += token['word'][2:]
    else:
        l += [ token ]
  return l


Vi prövar nu med KBs svenska modeller.


In [22]:
nertaggare = pipeline("ner", model="KB/bert-base-swedish-cased-ner", tokenizer='KB/bert-base-swedish-cased-ner')
svensktext = """Jag har köpt en ny mobil, en Samsung 10 E. Jag köpte den 17 april på Clas Ohlsson. Den kostade 8500 kronor. Jag ska ge den till Anton. """
output = retokenize(nertaggare(svensktext))
pd.DataFrame(output)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity,score,index,word,start,end
0,OBJ,0.798320,9,Samsung,29,36
1,TME,0.999616,15,den,53,56
2,TME,0.999537,16,17,57,59
3,TME,0.999568,17,april,60,65
4,PER,0.822165,19,Clas,69,73
5,PER,0.837884,20,Ohlsson,74,81
6,MSR,0.999865,24,8500,95,98
7,MSR,0.999752,26,kronor,100,106
8,PER,0.999645,33,Anton,128,133
